# Main children act outputs

### This code combines data created during the applications and disposals stages and creates the main children act csv, as well as the lookup file that feeds into the order/child count application and orders made tables

##### Import packages

In [ ]:
import os  # for file paths
import pandas as pd
import awswrangler as wr
import pydbtools as pydb  # see https://github.com/moj-analytical-services/pydbtools

# few things for viewing dataframes better
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)

In [ ]:
#publication periods are set in the main run file. However, if this code is run independantly they can be set here
#pub_year = 2022
#pub_qtr = 3

#### Creating main child act csv

In [ ]:
#Append main application and disposal data into one csv
print("creating main child act csv....")
pydb.create_temp_table( 
f"""
SELECT
  year,
  quarter AS qtr,
  type,
  count_type,
  public_private,
  disposal_type,
  order_type,
  order_type_code,
  gender,
  age_band,
  applicants_in_case,
  respondents_in_case,
  hc_indicator,
  count
FROM
  __temp__.ca_apps_csv
UNION ALL
SELECT
  year,
  quarter AS qtr,
  type,
  count_type,
  public_private,
  disposal_type,
  order_type,
  order_type_code,
  gender,
  age_band,
  applicants_in_case,
  respondents_in_case,
  hc_indicator,
  count
FROM 
  __temp__.ca_disp_csv
""",

"ca_csv")

#### Set as dataframe and export to s3

In [ ]:
ca_csv_data = pydb.read_sql_query("SELECT * FROM __temp__.ca_csv")

In [ ]:
ca_csv_df = pd.DataFrame(ca_csv_data)

In [ ]:
print("exporting main ca csv to s3....")
ca_csv_df.to_csv (r's3://alpha-family-data/fcsq_processing/children_act_temporary_outputs/CSV Children Act.csv', header = True, index = False)

#### Setting latest 4 quarterly periods

In [ ]:
#Table 3 and 4 only contain the last 4 quarterly periods, so here we set what those quarterly periods will be depending on the publication period
print("setting last 4 quarterly periods for order type tables....")
if(pub_qtr == 1):
    prev_q4 = 2
    prev_q3 = 3
    prev_q2 = 4
    prev_q1 = 1
    prev_yq4 = pub_year - 1
    prev_yq3 = pub_year - 1
    prev_yq2 = pub_year - 1
    prev_yq1 = pub_year
    
elif(pub_qtr == 2):
    prev_q4 = 3
    prev_q3 = 4
    prev_q2 = 1
    prev_q1 = 2
    prev_yq4 = pub_year - 1
    prev_yq3 = pub_year - 1
    prev_yq2 = pub_year
    prev_yq1 = pub_year
    
elif(pub_qtr == 3):
    prev_q4 = 4
    prev_q3 = 1
    prev_q2 = 2
    prev_q1 = 3
    prev_yq4 = pub_year - 1
    prev_yq3 = pub_year
    prev_yq2 = pub_year
    prev_yq1 = pub_year
    
elif(pub_qtr == 4):
    prev_q4 = 1
    prev_q3 = 2
    prev_q2 = 3
    prev_q1 = 4
    prev_yq4 = pub_year
    prev_yq3 = pub_year
    prev_yq2 = pub_year
    prev_yq1 = pub_year

#### Creating lookup file

In [ ]:
#Here we are creating columns that will form the lookup value that feeds in the order type/child count apps and disposals tables
print("creating lookup data table....")
pydb.create_temp_table( 
f"""
SELECT 
  CASE WHEN count_type IN ('Combined orders - children', 'Combined orders - orders')
        THEN '99'
       WHEN Order_type_code in (5,6,7)
        THEN '07'
       WHEN Order_type_code in (11,12)
        THEN '11' 
       WHEN Order_type_code in (15,16,17)
        THEN '15' 
       WHEN Order_type_code in (35,36)
        THEN '35'
    ELSE CAST(Order_type_code AS VARCHAR)
  END AS lookup_code,
  CASE WHEN Type = 'Disposal' THEN 'Order'
        ELSE Type
    END AS Type2,
  Public_private,  
  CASE WHEN Count_type = 'Combined orders - children' THEN 'Children'
       WHEN Count_type = 'Combined orders - orders' THEN 'Order type'
        ELSE Count_type
    END AS Count_type2,
  SUM(CASE WHEN year = 2011 THEN Count END) AS Y2011,
  SUM(CASE WHEN year = 2012 THEN Count END) AS Y2012,
  SUM(CASE WHEN year = 2013 THEN Count END) AS Y2013,
  SUM(CASE WHEN year = 2014 THEN Count END) AS Y2014,
  SUM(CASE WHEN year = 2015 THEN Count END) AS Y2015,
  SUM(CASE WHEN year = 2016 THEN Count END) AS Y2016,
  SUM(CASE WHEN year = 2017 THEN Count END) AS Y2017,
  SUM(CASE WHEN year = 2018 THEN Count END) AS Y2018,
  SUM(CASE WHEN year = 2019 THEN Count END) AS Y2019,
  SUM(CASE WHEN year = 2020 THEN Count END) AS Y2020,
  SUM(CASE WHEN year = 2021 THEN Count END) AS Y2021,
  SUM(CASE WHEN year = 2022 THEN Count END) AS Y2022,
  SUM(CASE WHEN Year = {prev_yq4} AND qtr = CAST({prev_q4} AS VARCHAR) THEN Count END) AS YQ1,
  SUM(CASE WHEN Year = {prev_yq3} AND qtr = CAST({prev_q3} AS VARCHAR) THEN Count END) AS YQ2,
  SUM(CASE WHEN Year = {prev_yq2} AND qtr = CAST({prev_q2} AS VARCHAR) THEN Count END) AS YQ3,
  SUM(CASE WHEN Year = {prev_yq1} AND qtr = CAST({prev_q1} AS VARCHAR) THEN Count END) AS YQ4
  
FROM
  __temp__.ca_csv
WHERE
  (count_type IN ('Children', 'Order type', 'Combined orders - children', 'Combined orders - orders')
   AND disposal_type NOT IN ('No order', 'Refused','Withdrawn'))  
GROUP BY
  CASE WHEN count_type IN ('Combined orders - children', 'Combined orders - orders')
        THEN '99'
       WHEN Order_type_code in (5,6,7)
        THEN '07'
       WHEN Order_type_code in (11,12)
        THEN '11' 
       WHEN Order_type_code in (15,16,17)
        THEN '15' 
       WHEN Order_type_code in (35,36)
        THEN '35'
    ELSE CAST(Order_type_code AS VARCHAR) END,
  CASE WHEN Type = 'Disposal' THEN 'Order'
        ELSE Type END,
  Public_private,
  CASE WHEN Count_type = 'Combined orders - children' THEN 'Children'
       WHEN Count_type = 'Combined orders - orders' THEN 'Order type'
        ELSE Count_type END;


""",
    
"lookup_cats")

In [ ]:
#Creating the lookup file
#Currently the latest year will need to be manually added - process probably won't get used in 2023
print("creating ca lookup file for order type tables....")
pydb.create_temp_table( 
f"""
SELECT 
   lookup_code||type2||public_private||count_type2 AS lookup,
   y2011,
   y2012,
   y2013,
   y2014,
   y2015,
   y2016,
   y2017,
   y2018,
   y2019,
   y2020,
   y2021,
   y2022,
   yq1,
   yq2,
   yq3,
   yq4
FROM
  __temp__.lookup_cats 

""",
    
"ca_lookup")

#### Export lookup to S3

In [ ]:
ca_lookup_data = pydb.read_sql_query("SELECT * FROM __temp__.ca_lookup")

In [ ]:
ca_lookup_df = pd.DataFrame(ca_lookup_data)

In [ ]:
print("exporting main ca lookup to s3....")
ca_lookup_df.to_csv (r's3://alpha-family-data/fcsq_processing/children_act_temporary_outputs/ca_lookup.csv', header = True, index=False)